In [1]:
import board
import busio
import time

In [2]:
def get_GPS_data(device_address=0x42, device_register=0xFF, time_limit=60, verbose=False):
    """This function pulls data from a Ublox ZED-F9P and returns an array of NMEA messages
    
    First the function connects to the GPS via I2C at the specified address and interal
    register address. For the ZED-F9P the default device address is 0x42 and the default
    internal register is at 0xFF. After collecting the bytes from the device, the bytes
    are processed into readable char. Data collection starts once a $ is found and ends
    when the byte returned is equivalent to 255.
    
    This function takes between 0.3 seconds to 2 seconds to collect the data. The bottle
    neck is the data acquisition on the device and changing the code did not significantly
    impact run time.
    
    Args:
        device_address: a hex number representing the device address. The default value is
            0x42.
        device_register: a hex representing the interal data register address. The default
            value is 0xFF.
        time_limit: an int or float that puts a limit on how long this function will run
            before it quits collecting data. This is to prevent an infinite loop in case
            the device hangs up.
        verbose: a boolean specifying whether to print progress and time
            
    Returns:
        device_output: an array of strings representing NMEA messages
    """
    i2c_hook = busio.I2C(board.SCL, board.SDA)
    while not i2c_hook.try_lock():
        pass
    
    buffer = bytearray(1)
    data_initialized = False
    data_finished = False
    data_array = []
    start_time = time.perf_counter()
    run_time = 0
    while((not data_initialized or not data_finished) and run_time < time_limit):
        try:
            i2c_hook.writeto(device_address, bytes([device_register]))
            i2c_hook.readfrom_into(device_address, buffer)
            char_arr = [chr(x) for x in buffer]
            cur_char = char_arr[0]
            data_length = len(data_array)
            if(cur_char == '$' and not data_initialized):
                if verbose:
                    print("Data Collection Started")
                data_initialized = True
            if((data_initialized and int(buffer[0]) == 255) and data_length > 1): 
                if verbose:
                    print("Data Collection Finished")
                data_finished = True
            
            if(data_initialized and not data_finished):
                data_array.append(cur_char)
                
        except OSError:
            continue
        cur_time = time.perf_counter()
        run_time = float(cur_time - start_time)
        
    if verbose:
        print(run_time)
    data_output = "".join(data_array).split("$")
    data_output = data_output[1:]
    
    return data_output

In [3]:
raw_gps = get_GPS_data()
for message in raw_gps:
    print(message)

GNRMC,082759.00,A,3725.60502,N,12209.13219,W,0.257,,250620,,,A,V*06

GNVTG,,T,,M,0.257,N,0.475,K,A*3B

GNGGA,082759.00,3725.60502,N,12209.13219,W,1,10,1.10,39.4,M,-30.0,M,,*42

GNGSA,A,3,26,31,03,16,04,22,,,,,,,1.98,1.10,1.65,1*05

GNGSA,A,3,70,80,71,72,73,,,,,,,,1.98,1.10,1.65,2*08

GNGSA,A,3,,,,,,,,,,,,,1.98,1.10,1.65,3*01

GNGSA,A,3,,,,,,,,,,,,,1.98,1.10,1.65,4*06

GPGSV,3,1,11,01,07,198,08,03,68,170,09,04,63,332,23,06,15,307,,1*6E

GPGSV,3,2,11,07,16,235,21,09,32,297,17,14,02,092,,16,43,119,26,1*61

GPGSV,3,3,11,22,45,162,10,26,38,068,21,31,12,050,16,1*50

GPGSV,3,1,11,01,07,198,,03,68,170,,04,63,332,,06,15,307,,*5F

GPGSV,3,2,11,07,16,235,,09,32,297,,14,02,092,,16,43,119,,*51

GPGSV,3,3,11,22,45,162,,26,38,068,,31,12,050,,*64

GLGSV,3,1,09,70,49,104,22,71,62,355,28,72,18,319,25,73,70,181,18,1*75

GLGSV,3,2,09,74,30,202,,80,38,035,20,86,00,255,,87,06,310,,1*70

GLGSV,3,3,09,88,01,345,,1*42

GLGSV,3,1,09,70,49,104,,71,62,355,,72,18,319,,73,70,181,,*40

GLGSV,3,2,09,74,30,202,,80,38,